In [ ]:
# -*- coding: utf-8 -*-
# pip install langgraph langchain-openai langchain-core gradio
# export OPENAI_API_KEY=...

from __future__ import annotations
from typing import TypedDict, Literal, Dict, Any, List, Optional
from dataclasses import dataclass
import json
import re

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableLambda
import gradio as gr

# =========================
#   LLM compartido
# =========================
llm_creative = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
llm_strict   = ChatOpenAI(model="gpt-4o", temperature=0.2)

# =========================
#   Prompt Maestro (GROWROUTINE)
# =========================
PROMPT_MAESTRO = """Eres el creador de contenido de @GrowRoutine. Tu tarea es generar, según el pedido del usuario, o bien:
A) Un CAROUSEL para Instagram (3 o 7 slides) o
B) Un EMAIL semanal motivacional, listo para enviar.

Contexto y estilo:
- Marca: @GrowRoutine
- Tono: claro, directo, motivacional, con frases contundentes y CTA amable.
- Idioma: español.
- Citas: siempre cita la fuente con formato: "📖 *Título* – Autor".
- Temas: extrae ideas de la base de aprendizajes (lista que te pasarán) sobre: Hábitos Atómicos (James Clear), Focus/Cómo ser un líder (Daniel Goleman), Controla tu tiempo (Brian Tracy), Los 7 hábitos (Stephen R. Covey), Empieza con el porqué (Simon Sinek), Lidera con gratitud (Gostick & Elton), etc.
- Lo que funcionó mejor: 
  1) Hook/frase potente al inicio. 
  2) Preguntas de introspección + micro-acción concreta (listas 3+3 cuando aplique). 
  3) Cierre con CTA que invite a seguir (@GrowRoutine) y a un reto semanal.
  4) En productividad, recuerda la Ley 888 cuando sea pertinente (8 trabajo / 8 descanso / 8 para ti).
  5) Copy conciso, con alto potencial de guardado/compartido.

Formato CAROUSEL:
- Si son 3 slides, usa esta estructura EXACTA:
  Slide 1 (fondo verde): frase potente (máx. 2–3 líneas).
  Slide 2 (fondo blanco): desarrollo breve y/o 2 preguntas + micro-acción “haz tu lista: 3 para más / 3 para menos” cuando aplique.
  Slide 3 (fondo foto): cierre emocional con invitación a seguir:
    "Creciendo 1% cada día… síguenos y recibe nuestro reto semanal."
    "@growroutine"
  Al final, entrega también **3 variantes de copy** para el post.
  Incluye siempre la cita del libro.

- Si son 7 slides, usa esta estructura EXACTA:
  1) Frase potente (verde)
  2) Contexto (blanco)
  3) Problema/tensión (blanco)
  4) Introspección con 2 preguntas (blanco)
  5) Acción práctica (blanco) – preferentemente listas 3+3 cuando aplique
  6) Motivación (verde)
  7) Cierre (foto): "Creciendo 1% cada día… síguenos y recibe nuestro reto semanal." + "@growroutine"
  Al final, 3 variantes de copy + cita del libro.

Formato EMAIL:
- Asunto breve con gancho (máx. 8–12 palabras).
- Apertura con idea-fuerza (1–2 líneas) y una cita de autoridad.
- 3–5 párrafos cortos y escaneables (líneas de 8–14 palabras).
- Sección “Esta semana haz esto:” con 2–4 pasos accionables.
- CTA final a etiquetar **@GrowRoutine** para recompartir.
- Firma: "Creciendo 1% cada día, [Tu Nombre] – @growroutine"

Reglas de calidad:
- Nada de jerga técnica; claridad > ornamento.
- Frases cortas, respirables, listas concisas.
- Siempre incluir la cita del libro.
- Nunca inventes autores/títulos. Basarte en la base provista.
- Mantén coherencia: propósito, hábitos pequeños, enfoque, gratitud, liderazgo empático, productividad sostenible.

Salida:
Devuelve SIEMPRE un JSON con:
{
  "type": "carousel" | "email",
  "source": {"title": "...", "author": "..."},
  "slides": [  // solo si type=carousel
    {"bg": "verde|blanco|foto", "text": "..."},
    ...
  ],
  "post_copy_variants": ["...", "...", "..."],  // para carousel
  "email": {  // solo si type=email
    "subject": "...",
    "body": "..."  // con saltos de línea listos para enviar
  }
}
Si el usuario dice “dame otro”, produce type="carousel". Si dice “dame otro correo", produce type="email".
Si no especifica slides, usa 3 por defecto. Si pide 7, usa el formato de 7.
Adecua el tema al histórico/base que recibas. Evita repetirte y rota fuentes.
"""

GEN_TEMPLATE = PromptTemplate.from_template(
    """{prompt_maestro}

### BASE_DE_APRENDIZAJES
{base_db}

### INSTRUCCIÓN_DEL_USUARIO
{user_input}

### PARÁMETROS
- slides: {slides}
- tema_preferente: {topic_hint}

### SALIDA
Devuelve SOLO el JSON indicado en las reglas."""
)

# =========================
#   Estado del grafo
# =========================
class State(TypedDict):
    user_input: str
    base_db: List[Dict[str, Any]]
    topic_hint: str
    req_type: Literal["carousel","email"]
    slides: int
    raw_json: str
    json_data: Dict[str, Any]
    validation_feedback: str
    final_json: str
    preview_md: str

# =========================
#   Utilidades
# =========================
def _infer_type(user_input: str) -> Literal["carousel", "email"]:
    ui = user_input.lower()
    if "correo" in ui or "email" in ui:
        return "email"
    return "carousel"

def _pick_slides(user_input: str) -> int:
    ui = user_input.lower()
    if "7" in ui or "siete" in ui:
        return 7
    return 3

def _base_to_text(base_db: List[Dict[str, Any]]) -> str:
    lines = []
    for it in base_db:
        fecha = it.get("fecha","")
        fuente = it.get("fuente","")
        nota = it.get("nota","")
        lines.append(f"- {fecha} | {fuente} | {nota}")
    return "\n".join(lines) if lines else "- (sin registros)"

def _try_force_json(s: str) -> Dict[str, Any]:
    try:
        return json.loads(s)
    except Exception:
        first = s.find("{")
        last = s.rfind("}")
        if first != -1 and last != -1 and last > first:
            return json.loads(s[first:last+1])
        raise

# =========================
#   Nodos del grafo
# =========================
def router_node(state: State) -> State:
    req_type = _infer_type(state["user_input"])
    slides = _pick_slides(state["user_input"])
    state["req_type"] = req_type
    state["slides"] = slides
    return state

def generator_node(state: State) -> State:
    base_text = _base_to_text(state["base_db"])
    topic_hint = state.get("topic_hint") or "productividad y enfoque"

    prompt = GEN_TEMPLATE.format(
        prompt_maestro=PROMPT_MAESTRO,
        base_db=base_text,
        user_input=state["user_input"],
        slides=state["slides"],
        topic_hint=topic_hint
    )

    raw = (GEN_TEMPLATE | llm_creative | StrOutputParser()).invoke({
        "prompt_maestro": PROMPT_MAESTRO,
        "base_db": base_text,
        "user_input": state["user_input"],
        "slides": state["slides"],
        "topic_hint": topic_hint
    })
    state["raw_json"] = raw
    state["json_data"] = _try_force_json(raw)
    return state

def validator_node(state: State) -> State:
    data = state["json_data"]
    errors: List[str] = []

    # Regla básica de tipo
    t = data.get("type")
    if t not in ("carousel", "email"):
        errors.append("El campo 'type' debe ser 'carousel' o 'email'.")

    # Cita (source)
    src = data.get("source") or {}
    if not (src.get("title") and src.get("author")):
        errors.append("Falta 'source' con 'title' y 'author'.")

    # Validación específica por tipo
    if t == "carousel":
        slides = data.get("slides")
        if not isinstance(slides, list) or not slides:
            errors.append("Carousel debe incluir 'slides' (lista).")
        else:
            # Validar estructura 3 o 7
            if state["slides"] == 3 and len(slides) != 3:
                errors.append("Carousel solicitado de 3 slides, pero no devuelve 3.")
            if state["slides"] == 7 and len(slides) != 7:
                errors.append("Carousel solicitado de 7 slides, pero no devuelve 7.")
            # Validar bg permitido
            for i, s in enumerate(slides, start=1):
                if s.get("bg") not in ("verde","blanco","foto"):
                    errors.append(f"Slide {i} con 'bg' inválido (usa verde|blanco|foto).")
                if not s.get("text"):
                    errors.append(f"Slide {i} sin 'text'.")
        if not data.get("post_copy_variants") or len(data["post_copy_variants"]) != 3:
            errors.append("Faltan exactamente 3 'post_copy_variants'.")

    if t == "email":
        email = data.get("email") or {}
        if not email.get("subject"):
            errors.append("Email sin 'subject'.")
        if not email.get("body"):
            errors.append("Email sin 'body'.")

    # Verificar cita visible en el cuerpo/cierre
    # Permitimos que esté solo en 'source', pero si es email, alentamos incluirlo:
    if t == "email":
        body = (data.get("email") or {}).get("body","")
        if "📖" not in body:
            errors.append("El email debe incluir explícitamente la cita con '📖' en el body.")

    # Resultado
    if errors:
        fb = "No aprobado\nFeedback:\n- " + "\n- ".join(errors)
    else:
        fb = "Aprobado\nFeedback:\n- Cumple las reglas principales."

    state["validation_feedback"] = fb
    return state

def should_regenerate(state: State) -> str:
    fb = state.get("validation_feedback","").lower()
    return "Generate" if "no aprobado" in fb else "Finalize"

def improve_and_regenerate(state: State) -> State:
    """Reinyecta feedback para corregir y regenerar una vez."""
    base_text = _base_to_text(state["base_db"])
    topic_hint = state.get("topic_hint") or "productividad y enfoque"

    improve_clause = f"\n\n### FEEDBACK A CORREGIR (OBLIGATORIO)\n{state['validation_feedback']}\nAjusta la salida y devuelve SOLO el JSON.\n"

    raw = (PromptTemplate.from_template(
        "{pm}\n\n### BASE_DE_APRENDIZAJES\n{base}\n\n### INSTRUCCIÓN_DEL_USUARIO\n{ui}\n\n### PARÁMETROS\n- slides: {slides}\n- tema_preferente: {topic}\n{fix}\n\n### SALIDA\nDevuelve SOLO el JSON indicado en las reglas."
    ) | llm_strict | StrOutputParser()).invoke({
        "pm": PROMPT_MAESTRO,
        "base": base_text,
        "ui": state["user_input"],
        "slides": state["slides"],
        "topic": topic_hint,
        "fix": improve_clause
    })

    state["raw_json"] = raw
    state["json_data"] = _try_force_json(raw)
    return state

def final_pack_node(state: State) -> State:
    data = state["json_data"]
    state["final_json"] = json.dumps(data, ensure_ascii=False, indent=2)

    # Vista previa Markdown (simple)
    if data.get("type") == "carousel":
        slides = data.get("slides", [])
        md = ["# 🔄 Preview – Carousel"]
        for i, s in enumerate(slides, start=1):
            md.append(f"### Slide {i} ({s.get('bg','?')})")
            md.append(s.get("text","").strip())
            md.append("")
        copies = data.get("post_copy_variants", [])
        if copies:
            md.append("### Variantes de copy")
            for c in copies:
                md.append(f"- {c}")
        src = data.get("source", {})
        md.append(f"\n**Cita:** 📖 *{src.get('title','?')}* – {src.get('author','?')}")
        state["preview_md"] = "\n".join(md)
    else:
        email = data.get("email", {})
        md = [
            "# ✉️ Preview – Email",
            f"**Asunto:** {email.get('subject','')}",
            "",
            email.get("body",""),
            "",
        ]
        src = data.get("source", {})
        md.append(f"**Cita:** 📖 *{src.get('title','?')}* – {src.get('author','?')}")
        state["preview_md"] = "\n".join(md)

    return state

# =========================
#   Construcción del grafo
# =========================
builder = StateGraph(State)
builder.add_node("Route", RunnableLambda(router_node))
builder.add_node("Generate", RunnableLambda(generator_node))
builder.add_node("Validate", RunnableLambda(validator_node))
builder.add_node("Improve", RunnableLambda(improve_and_regenerate))
builder.add_node("Finalize", RunnableLambda(final_pack_node))

builder.set_entry_point("Route")
builder.add_edge("Route", "Generate")
builder.add_edge("Generate", "Validate")
builder.add_conditional_edges("Validate", should_regenerate, {
    "Generate": "Improve",
    "Finalize": "Finalize"
})
builder.add_edge("Improve", "Validate")
builder.add_edge("Finalize", END)

graph = builder.compile(checkpointer=MemorySaver())


KeyboardInterrupt: 

In [ ]:

# =========================
#   Función de alto nivel
# =========================
def growroutine_generate(
    user_input: str,
    base_db: List[Dict[str, Any]],
    topic_hint: str = "productividad y enfoque",
    thread_id: str = "growroutine-run"
) -> Dict[str, Any]:
    """
    - user_input: "dame otro" | "dame otro 7" | "dame otro correo" | etc.
    - base_db: lista de dicts con {fecha, fuente, nota}
    - topic_hint: pista de tema preferente
    Devuelve dict con:
      - final_json (str)
      - preview_md (str)
      - validation_feedback (str)
    """
    result = graph.invoke(
        {
            "user_input": user_input,
            "base_db": base_db,
            "topic_hint": topic_hint
        },
        config={"configurable": {"thread_id": thread_id}}
    )
    return {
        "final_json": result.get("final_json",""),
        "preview_md": result.get("preview_md",""),
        "validation_feedback": result.get("validation_feedback","")
    }

# =========================
#   Ejemplo rápido (CLI)
# =========================
if __name__ == "__main__":
    base_ejemplo = [
        {"fecha":"2024-11-04","fuente":"Hábitos Atómicos – James Clear","nota":"Regla de los 2 minutos, diseñar entorno, señales y fricción."},
        {"fecha":"2025-03-01","fuente":"Controla tu tiempo, controla tu vida – Brian Tracy","nota":"Listas 3+3, foco en lo importante, Ley 888."},
        {"fecha":"2024-12-27","fuente":"Focus – Daniel Goleman","nota":"Atención como músculo; evitar multitarea; descanso mental."},
        {"fecha":"2024-12-15","fuente":"Cómo ser un líder – Daniel Goleman","nota":"Empatía cognitiva/emocional; estilos de liderazgo."},
        {"fecha":"2025-02-23","fuente":"Empieza con el porqué – Simon Sinek","nota":"Propósito claro como motor."},
        {"fecha":"2025-03-02","fuente":"Lidera con gratitud – Gostick & Elton","nota":"Reconocimiento específico y oportuno."},
    ]

    # Test: Carousel por defecto (3)
    out1 = growroutine_generate("dame otro", base_ejemplo, topic_hint="productividad")
    print("=== VALIDATION ===")
    print(out1["validation_feedback"])
    print("=== JSON ===")
    print(out1["final_json"])

    # Test: Email
    out2 = growroutine_generate("dame otro correo", base_ejemplo, topic_hint="propósito")
    print("=== VALIDATION ===")
    print(out2["validation_feedback"])
    print("=== JSON ===")
    print(out2["final_json"])

# =========================
#   UI Gradio (opcional)
# =========================
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 GrowRoutine Agent – Carousel / Email Generator")
    with gr.Row():
        user_in = gr.Textbox(label="🗣️ Instrucción (ej: 'dame otro', 'dame otro 7', 'dame otro correo')", lines=1)
        topic_in = gr.Textbox(label="🎯 Topic hint (opcional)", value="productividad y enfoque")
    base_in = gr.Textbox(
        label="📚 Base de aprendizajes (JSON list de objetos {fecha, fuente, nota})",
        value=json.dumps([
            {"fecha":"2024-11-04","fuente":"Hábitos Atómicos – James Clear","nota":"Regla de los 2 minutos, diseñar entorno, señales y fricción."},
            {"fecha":"2025-03-01","fuente":"Controla tu tiempo, controla tu vida – Brian Tracy","nota":"Listas 3+3, foco en lo importante, Ley 888."},
            {"fecha":"2024-12-27","fuente":"Focus – Daniel Goleman","nota":"Atención como músculo; evitar multitarea; descanso mental."},
            {"fecha":"2024-12-15","fuente":"Cómo ser un líder – Daniel Goleman","nota":"Empatía cognitiva/emocional; estilos de liderazgo."},
            {"fecha":"2025-02-23","fuente":"Empieza con el porqué – Simon Sinek","nota":"Propósito claro como motor."},
            {"fecha":"2025-03-02","fuente":"Lidera con gratitud – Gostick & Elton","nota":"Reconocimiento específico y oportuno."},
        ], ensure_ascii=False, indent=2),
        lines=12
    )

    run_btn = gr.Button("🚀 Generar")
    feedback_out = gr.Textbox(label="✅ Validación", lines=8)
    json_out = gr.Code(label="📦 JSON Final", language="json")
    preview_out = gr.Markdown(label="🖼️ Vista previa")

    def _run(user_text, topic_hint, base_json):
        try:
            base_db = json.loads(base_json)
            out = growroutine_generate(user_text, base_db, topic_hint)
            return out["validation_feedback"], out["final_json"], out["preview_md"]
        except Exception as e:
            return f"Error: {e}", "{}", "Error al generar vista previa."

    run_btn.click(_run, [user_in, topic_in, base_in], [feedback_out, json_out, preview_out])

demo.launch(share=True)
